# グラフを作成するサンプルコード

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from glob import glob
from tqdm import tqdm
import japanize_matplotlib

import warnings
warnings.filterwarnings('ignore')

# CSVのカラム名を定義
column_names = [
    'Sec1',
    'Cycle1',
    'Phase1',
    'Qmax1',
    'DATE',
    'TIME',
    'Freq',
    'V1',
    'f0',
    'BW'
]

# ファイルパスを取得
target_dir = "C:\\SOKEN\\PD9\\SampleSoftware260\\Data\\pd_temp"    # グラフ化するフォルダを指定
flist = glob(f"{target_dir}\\*csv")
flist.sort()

for fpath in tqdm(flist):
    # CSVを読み込む
    data = pd.read_csv(fpath, names=column_names, header=None, index_col=False)
    data = data.dropna(subset=['Phase1', 'Qmax1'])
    # print(data.info())

    # 散布図のデータ
    x = data['Phase1']
    y = data['Qmax1']

    # 正弦波の計算
    x_sin = np.linspace(0, 360, 1000)
    y_min = min(data['Qmax1'])
    y_max = max(data['Qmax1'])

    y1 = (y_max - y_min) / 2 * np.sin(np.radians(x_sin)) + (y_max + y_min) / 2
    y2 = (y_max - y_min) / 2 * np.sin(np.radians(x_sin) + np.radians(120)) + (y_max + y_min) / 2
    y3 = (y_max - y_min) / 2 * np.sin(np.radians(x_sin) + np.radians(240)) + (y_max + y_min) / 2

    # 密度推定
    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)

    # 散布図のプロット
    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, c=z, s=5, edgecolor=None, cmap='jet')

    # 正弦波の描画
    plt.plot(x_sin, y1, label='位相差0度')
    plt.plot(x_sin, y2, label='位相差60度')
    plt.plot(x_sin, y3, label='位相差120度')

    plt.colorbar(label='密度')
    plt.xlabel('位相φ')
    plt.ylabel('電荷量Q')
    plt.xticks(np.arange(0, 361, step=60))
    plt.title('Q-φ Characteristic Graph')
    plt.savefig(f'test_img\\{fpath.split("\\")[-1]}.png')
    print(f"Save Figure: test_img\\{fpath.split("\\")[-1]}.png")
